In [1]:
from MultiEurlexDataset import MultiEurlexDataset
from MultiEvalDataset import MultiEvalDataset
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline, EarlyStoppingCallback
from torch.utils.data import RandomSampler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import numpy as np
import torch.nn as nn

In [2]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
device='cuda'
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base', padding=True, trunaction=True)
test_dataset = MultiEurlexDataset(split='test',languages = ['hr'], tokenizer=tokenizer)
test_dataset_readable = MultiEvalDataset(split='test',languages = ['hr'])

Reusing dataset multi_eurlex (/home/mrajnovic/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


Reusing dataset multi_eurlex (/home/mrajnovic/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0000% of dataset loaded
10.0000% of dataset loaded
20.0000% of dataset loaded
30.0000% of dataset loaded
40.0000% of dataset loaded
50.0000% of dataset loaded
60.0000% of dataset loaded
70.0000% of dataset loaded
80.0000% of dataset loaded
90.0000% of dataset loaded


In [4]:
ticks = ["social questions",
         "industry",
         "finance",
         "trade",
         "business and competition",
         "international relations",
         "agriculture, forestry and fisheries",
         "production, technology and research",
         "transport",
         "employment and working conditions",
         "politics",
         "law",
         "education and communications",
         "international organisations",
         "energy",
         "EUROPEAN UNION",
         "science",
         "agri-foodstuffs",
         "geography",
         "economics",
         "environment"]

In [5]:
metric = f1_score

def compute_metrics(eval_pred):
    label_pred, label_true = eval_pred
    sigmoid = nn.Sigmoid()
    label_pred=sigmoid(torch.tensor(label_pred)).numpy()
    label_pred[label_pred < 0.5] = 0
    label_pred[label_pred >= 0.5] = 1
    result={}
    result['f1'] = metric(label_true,label_pred, average='samples')
    result['classification_report'] = classification_report(label_true,label_pred)
    result['confusion_matrix'] = confusion_matrix(label_true, label_pred)
    return result

In [6]:
model = XLMRobertaForSequenceClassification.from_pretrained('/home/mrajnovic/EuroVoc_project/results/slavic_croatian/checkpoint-72795', num_labels=21, problem_type="multi_label_classification", )

In [7]:
print(test_dataset_readable[17][0])
print()
print('Labels:')
for tick in test_dataset_readable[17][1]:
    print(ticks[tick])

32013D0131
L 076/1
SLUŽBENI LIST EUROPSKE UNIJE
ODLUKA KOMISIJE
od 4. ožujka 2013.
o donošenju priručnika za korisnike kojim se utvrđuju koraci koje je potrebno poduzeti za sudjelovanje u sustavu EMAS u skladu s Uredbom (EZ) br. 1221/2009 Europskog parlamenta i Vijeća o dobrovoljnom sudjelovanju organizacija u sustavu upravljanja okolišem i neovisnog ocjenjivanja Zajednice (EMAS)
(priopćena pod brojem dokumenta C(2013) 1114)
(Tekst značajan za EGP)
(2013/131/EU)
EUROPSKA KOMISIJA,
uzimajući u obzir Ugovor o funkcioniranju Europske unije,
uzimajući u obzir Uredbu (EZ) br. 1221/2009 Europskog parlamenta i Vijeća od 25. studenoga 2009. o dobrovoljnom sudjelovanju organizacija u sustavu upravljanja okolišem i neovisnog ocjenjivanja Zajednice (EMAS) te stavljanju izvan snage Uredbe (EZ) br. 761/2001 i odluka Komisije 2001/681/EZ i 2006/193/EZ (1), a posebno njezin članak 46. stavak 5.,
budući da:
Poduzeća i druge organizacije trebaju dobiti dodatne informacije i smjernice o koracima koje je

In [17]:
sigmoid = nn.Sigmoid()
logits = sigmoid(model(input_ids=test_dataset[17]['input_ids'].unsqueeze(0), attention_mask=test_dataset[17]['attention_mask'].unsqueeze(0))['logits'])

In [33]:
logits[logits < 0.5] = 0
logits[logits > 0.5] = 1
guesses = np.where(logits[0].detach().cpu().numpy() == 1)[0].tolist()

In [34]:
guesses

[12, 20]

In [35]:
print('Labels:')
for tick in guesses:
    print(ticks[tick])

Labels:
education and communications
environment
